In [1]:
from keras.datasets import mnist
from keras.layers import Dense, Dropout
from keras.optimizers import SGD, Adam
from keras.callbacks import LearningRateScheduler
from keras.models import Model, Sequential
from keras.initializers import RandomNormal
from keras.engine.topology import Layer, InputSpec
import keras.backend as K
from sklearn.cluster import KMeans
import numpy as np
from keras.utils import normalize
from sklearn.utils.linear_assignment_ import linear_assignment

Using TensorFlow backend.


In [0]:
class Autoencoder:
    def __init__(self, batch_size=256, nb_epoch_layerwise=250,
                 nb_epoch_finetune=500,
                 dropout_rate=0.2,
                 init_stddev=0.01,
                 learning_rate=0.1,
                 lr_epoch_update=200,
                 autoencoder_layers=[784, 500, 500, 2000, 10]):
        
        self.batch_size = batch_size
        self.nb_epoch_layerwise = nb_epoch_layerwise
        self.nb_epoch_finetune = nb_epoch_finetune
        self.dropout_rate = dropout_rate
        self.init_stddev = init_stddev
        self.learning_rate = learning_rate
        self.lr_epoch_update = lr_epoch_update  # TODO change
        self.autoencoder_layers = autoencoder_layers

    def lr_update(self, epoch):
        initial_rate = self.learning_rate
        factor = int(epoch / self.lr_epoch_update)
        lr = initial_rate / (10 ** factor)
        return lr

    def get_lr_scheduler(self):
        return LearningRateScheduler(self.lr_update)

    def layerwise_pretraining(self, X):
        aes = []
        X_tmp = np.copy(X)
        for i, (n_in, n_out) in enumerate(zip(self.autoencoder_layers[:-1], self.autoencoder_layers[1:]), start=1):
            print('Training the layer {}: Input {} -> Output {}'.format(i, n_in, n_out))
            encoder_activation = 'linear' if i == (len(self.autoencoder_layers) - 1) else 'relu'
            decoder_activation = 'linear' if i == 1 else 'relu'

            encoder = Sequential([Dropout(self.dropout_rate), Dense(n_out, input_shape=(n_in,), activation=encoder_activation,
                                                                    kernel_initializer=RandomNormal(mean=0.0,
                                                                                                    stddev=self.init_stddev,
                                                                                                    seed=None),
                                                                    bias_initializer='zeros')])
            decoder = Sequential([Dropout(self.dropout_rate), Dense(n_in, input_shape=(n_out,), activation=decoder_activation,
                                                                    kernel_initializer=RandomNormal(mean=0.0,
                                                                                                    stddev=self.init_stddev,
                                                                                                    seed=None),
                                                                    bias_initializer='zeros')])
            ae = Sequential([encoder, decoder])
            ae.compile(loss='mse', optimizer=SGD(lr=self.learning_rate, decay=0, momentum=0.9))
            ae.fit(X_tmp, X_tmp, batch_size=self.batch_size, epochs=self.nb_epoch_layerwise,
                   callbacks=[self.get_lr_scheduler()])
            aes.append(ae)
            X_tmp = encoder.predict(X_tmp)
            print(f'Output shape: {X_tmp.shape}')
        return aes

    def finetuning(self, aes, X):
        decoders_wout_dropout = []
        encoders_wout_dropout = []
        for i, (n_in, n_out) in enumerate(zip(self.autoencoder_layers[:-1], self.autoencoder_layers[1:])):
            print(f'ITERATION: {i+1}, X.shape={X.shape}')
            encoder_activation = 'linear' if i == (len(self.autoencoder_layers) - 1) else 'relu'
            decoder_activation = 'linear' if i == 1 else 'relu'

            encoder_weights = aes[i].layers[0].get_weights()
            decoder_weights = aes[i].layers[1].get_weights()

            encoder = Sequential([Dense(n_out, input_shape=(n_in,),
                                        activation=encoder_activation, weights=encoder_weights)])
            decoder = Sequential([Dense(n_in, input_shape=(n_out,),
                                        activation=decoder_activation, weights=decoder_weights)])

            encoders_wout_dropout.append(encoder)
            decoders_wout_dropout.append(decoder)

        decoders_wout_dropout.reverse()
        autoencoder = Sequential(encoders_wout_dropout + decoders_wout_dropout)

        autoencoder.compile(loss='mse', optimizer=SGD(lr=self.learning_rate, decay=0, momentum=0.9))
        # finetune autoencoder
        autoencoder.fit(X, X, batch_size=self.batch_size, epochs=self.nb_epoch_finetune,
                        callbacks=[self.get_lr_scheduler()])

        encoder = Sequential(encoders_wout_dropout)
        # update weights
        for i in range(len(self.autoencoder_layers) - 1):
            encoder.layers[i].set_weights(autoencoder.layers[i].get_weights())

        return autoencoder, encoder


def save_model(model, filename):
    !pip install -U -q PyDrive
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials

    # 1. Authenticate and create the PyDrive client.
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)

    # 2. Save Keras Model or weights on google drive

    # create on Colab directory
    model.save(filename)
    model_file = drive.CreateFile({'title': filename})
    model_file.SetContentFile(filename)
    model_file.Upload()

    # download to google drive
    drive.CreateFile({'id': model_file.get('id')})

In [0]:
class ClusteringLayer(Layer):

    def __init__(self, n_clusters, weights=None, alpha=1.0, **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(ClusteringLayer, self).__init__(**kwargs)
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.initial_weights = weights
        self.input_spec = InputSpec(ndim=2)

    def build(self, input_shape):
        assert len(input_shape) == 2
        input_dim = input_shape[1]
        self.input_spec = InputSpec(dtype=K.floatx(), shape=(None, input_dim))
        self.clusters = self.add_weight((self.n_clusters, input_dim), initializer='glorot_uniform', name='clusters')
        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        self.built = True

    def call(self, inputs, **kwargs):
        q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1))
        return q

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 2
        return input_shape[0], self.n_clusters

In [0]:
class DeepEmbeddedClustering:
    def __init__(self,
                 encoder,
                 n_clusters=10,
                 alpha=1.0):
      
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.encoder = encoder
        self.batch_size = 256

        # init DEC model
        clustering_layer = ClusteringLayer(self.n_clusters, name='clustering')
        self.model =  Sequential([self.encoder, clustering_layer])

    def target_distribution(self, q):
        weight = q ** 2 / q.sum(0)
        return (weight.T / weight.sum(1)).T

    def predict(self, X):  
        q = self.model.predict(X, verbose=0)
        return q.argmax(1)

    def fit_predict(self, X, tolerance=0.001):
        self.model.compile(optimizer=SGD(lr=0.01, momentum=0.9), loss='kld')
        # initialize cluster centres by use of k-means
        kmeans = KMeans(n_clusters=self.n_clusters, n_init=20)
        y_pred = kmeans.fit_predict(self.encoder.predict(X))
        y_pred_last = np.copy(y_pred)
        self.model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])

        iteration = 0
        index = 0
        index_array = np.arange(X.shape[0])
        interval = 10
        while iteration < 10 ** 4 and delta > tolerance:
          if iteration % interval == 0:
            q = self.model.predict(X)
            p = self.target_distribution(q)  # update target distribution p
            y_pred = q.argmax(1)
            delta = ((y_pred != y_pred_last).sum().astype(np.float32) / y_pred.shape[0])


          idx = index_array[index * self.batch_size: min((index + 1) * self.batch_size, X.shape[0])]
          loss = self.model.train_on_batch(x=X[idx], y=p[idx])
          print(f'ITERATION: {iteration}; LOSS: {loss}')
          print(f'TOLERANCE: {delta}')
          index = index + 1 if (index + 1) * self.batch_size <= X.shape[0] else 0

          y_pred_last = np.copy(y_pred)
          iteration += 1
        return y_pred

def cluster_accuracy(y_true, y_pred):
  D = 10
  w = np.zeros((D, D), dtype=np.int64)
  for i in range(y_pred.size):
    w[y_pred[i], y_true[i]] += 1
  ind = linear_assignment(- w)
  return sum([w[i, j] for i, j in ind])*1.0 / y_pred.size


In [5]:
# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print(X_train.shape)
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0

11493376/11490434 [==============================] - 9s 1us/step
(60000, 28, 28)


In [0]:
autoencoder = Autoencoder()
aes=autoencoder.layerwise_pretraining(X_train)
ae, encoder=autoencoder.finetuning(aes, X_train)
save_model(ae, 'autoencoder.h5')
save_model(encoder, 'encoder.h5')

Training the layer 1: Input 784 -> Output 500
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/250
60000/60000 [==============================] - 4s 70us/step - loss: 0.0670
Epoch 2/250
60000/60000 [==============================] - 1s 22us/step - loss: 0.0450
Epoch 3/250
60000/60000 [==============================] - 1s 22us/step - loss: 0.0369
Epoch 4/250
60000/60000 [==============================] - 1s 22us/step - loss: 0.0326
Epoch 5/250
60000/60000 [==============================] - 1s 20us/step - loss: 0.0296
Epoch 6/250
60000/60000 [==============================] - 1s 21us/step - loss: 0.0274
Epoch 7/250
60000/60000 [==============================] - 1s 20us/step - loss: 0.0258
Epoch 8/250
60000/60000 [==============================] - 1s 21us/step - loss: 0.0246
Epoch 9/250
60000/60000

In [0]:
ae.summary()

In [0]:
encoder.summary()

In [0]:
dec = DeepEmbeddedClustering(encoder)
y_dec = dec.fit_predict(X_train)

# RESULTS:

In [0]:
kmeans = KMeans(n_clusters=10, n_init=20)
y_pred = kmeans.fit_predict(X_train)
print(cluster_acc(y_train, y_pred))

In [0]:
kmeans = KMeans(n_clusters=10, n_init=20)
y_pred = kmeans.fit_predict(encoder.predict(X_train))
print(cluster_acc(y_train, y_pred))

In [0]:
cluster_accuracy(y_train, y_dec)